### Overview of Multiclass Classification: **one-vs-all** and **all-pairs**

Give an overview of the algorithm and describe its advantages and disadvantages.

In this final project, we will be using the UCI Iris dataset we encountered in our previous homework:\
[`https://archive.ics.uci.edu/dataset/53/iris`](https://archive.ics.uci.edu/dataset/53/iris)

What we will be comparing to:
[scikit-learn multiclass classification](https://scikit-learn.org/1.5/modules/multiclass.html#multilabel-classification)


### Representation: Logistic Regression

describe how the feature values are converted into a single number prediction.

#### Binary Logistic Regression
Given sample's feature values $x \in \mathbb{R}^{d}$ and a label $y  \in \{-1, 1\}$, classification of input $x$ is predicted through combination of affine function and sigmoid function.
$$ \langle w, x\rangle + b = y \text{  and  } \sigma = \frac{1}{1 + e^{-y}}$$

#### Multiclass Logistic Regression
one-vs-all

all-pairs


### Loss: Logistic Loss

describe the metric used to measure the difference between the model’s prediction and the target variable.

### Optimizer

describe the numerical algorithm used to find the model parameters that minimize the loss for a training set.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split